In [1]:
import sys
sys.path.append("/home/yamato/Application/reduction_tools/")
import reduction_utils_mpi as utils
import casatasks
import casatools
from linedictionary import line_dict
from diskdictionary import disk_dict
import shutil
import os
import numpy as np
import analysis_utils as au
import matplotlib.pyplot as plt
from astropy.convolution import convolve_fft
from scipy import optimize
import subprocess

%matplotlib widget

tb = casatools.table()
ia = casatools.image()

def rad2arcsec(val):
    return val * 180 * 3600 / np.pi

imagename = "taper_test_robust"
vis = "./data/measurement_set/MWC_480_continuum.ms/"
weighting = "briggs"
robust = 0.5
cellsize = "0.01arcsec"
imsize = utils.get_imsize(vis=vis, field="MWC_480", cellsize=cellsize)

You appear to be importing analysisUtils into python (not CASA). version =  3.10.5
CASAPATH is not defined, so I am skipping a lot of imports


In [2]:
def get_beam_info(imagename):
    ia.open(imagename)
    beam = ia.summary()["restoringbeam"]
    ia.close()

    bmaj = beam["major"]["value"]
    bmin = beam["minor"]["value"]
    bpa = beam["positionangle"]["value"]

    return bmaj, bmin, bpa

def get_xy_1D_coord(psf, imsize):
    peak_x, peak_y = np.unravel_index(np.argmax(psf), psf.shape)
    x = -(np.arange(imsize) - peak_x)
    y = np.arange(imsize) - peak_y
    return x, y

def get_xy_2D_coord(psf, imsize):
    x, y = get_xy_1D_coord(psf, imsize)
    xx, yy = np.meshgrid(x, y, indexing="ij") # indexing="ij" is needed to match the axis order to that of casa image
    return xx, yy

def rotate_coord(x, y, theta):
    rotangle = np.deg2rad(theta)
    x_rot = x * np.cos(rotangle) + y * np.sin(rotangle)
    y_rot = x * np.sin(rotangle) - y * np.cos(rotangle)
    return x_rot, y_rot

def Gaussian2D(x, y, sigma_x, sigma_y, PA):
    theta = 90. - PA # from negative x to positive y
    x_rot, y_rot = rotate_coord(x, y, theta)
    g = 1.*np.exp(-((x_rot/sigma_x)**2/2. + (y_rot/sigma_y)**2/2.))
    return g

def get_nulled_psf(psf_array, r, theta, ntheta=24):

    # azimuthal coordinate
    da = 2 * np.pi / ntheta
    azim = np.arange(-np.pi, np.pi, da)

    nulled_psf = psf_array.copy()

    for a in azim:
        # some overlapping azimuth: see https://github.com/MPoL-dev/MPoL/blob/main/src/mpol/gridding.py#L441
        wedge = (theta >= a - 0.3 * da) & (theta <= a + 1.3 * da)
        wedge_is_neg = wedge & (psf_array < 0)
        first_null = np.min(r[wedge_is_neg])
        nulled_psf[wedge & (r >= first_null)] = 0.0

    return nulled_psf

def calc_JvM_epsilon(xx, yy, target_beam_array, tapered_psf_array):
    r, theta = np.sqrt(xx**2 + yy**2), np.arctan2(yy, xx)
    epsilon = np.sum(target_beam_array) / np.sum(get_nulled_psf(tapered_psf_array, r, theta))
    print("JvM epsilon: {:.4f}".format(epsilon))
    return epsilon

def calc_uvtaper(vis, imagename, field="", specmode="mfs", weighting="superuniform", cellsize="0.1arcsec", imsize=None, target_beam=(0.1, 0.1, 0.0), **tclean_kwargs):
    if not isinstance(target_beam, (tuple, list)):
        target_beam = (target_beam, target_beam, 0.0)
    target_bmaj, target_bmin, target_bpa = target_beam

    print("Starting to estimate the uvtaper which achieve the target beam of {:.4f} arcsec x {:.4f} arcsec (P.A. = {:.4f} deg)".format(*target_beam))

    # calculate image size
    if imsize is None:
        imsize = utils.get_imsize(vis=vis, field=field, cellsize=cellsize)
    
    print(f"Pixel size: {cellsize}")
    print(f"Image size: {imsize}")
    dpix = float(cellsize.replace("arcsec", ""))

    # calculate and fit psf
    msg = "Calculating and fitting PSF with the original weighting scheme (weighting = {:s}".format(weighting)
    if "briggs" in weighting:
        robust = tclean_kwargs.get("robust", 0.5)
        msg += f", robust = {robust}"
    msg += ")"
    print(msg)

    for ext in [".psf", ".pb", ".sumwt"]:
        if os.path.exists(imagename + ext):
            shutil.rmtree(imagename + ext)
    casatasks.tclean(
        vis=vis,
        imagename=imagename,
        specmode=specmode,
        weighting=weighting,
        cell=cellsize,
        imsize=imsize,
        calcres=False,
        calcpsf=True,
        **tclean_kwargs
    )
    print("Done.")

    # get the original psf array
    ia.open(imagename + ".psf")
    psf_orig = ia.getregion().squeeze()
    ia.close()
    
    # fetch beam info
    bmaj, bmin, bpa = get_beam_info(imagename + ".psf")
    print(
        "Restoring beam shape: {:.4f} arcsec x {:.4f} arcsec (P.A. = {:.4f} deg)".format(
            bmaj, bmin, bpa
        )
    )

    # window out the central region for computational efficiency while keeping enough region for convolution to work well
    print("Windowing out the central region for computational efficiency")
    window_factor = 20 # factor to determined the window; multiplied to the target major axis
    start_idx = int(imsize*0.5) - int(target_bmaj * window_factor / dpix)
    end_idx = int(imsize*0.5) + int(target_bmaj * window_factor / dpix)
    psf_windowed = psf_orig[start_idx:end_idx, start_idx:end_idx]
    imsize_windowed = psf_windowed.shape[0]
    print(f"Image size of windowed psf: {imsize_windowed}")

    # minimizing the metric
    xx, yy = get_xy_2D_coord(psf_windowed, imsize_windowed)
    target_beam = Gaussian2D(xx, yy, sigma_x=au.FWHM_to_sigma(target_bmaj/dpix), sigma_y=au.FWHM_to_sigma(target_bmin/dpix), PA=target_bpa)
    psf_cutoff = tclean_kwargs.get("psf_cutoff", 0.35)

    def beam_chi2(params):
        tmaj, tmin, tpa = params

        kernel = Gaussian2D(xx, yy, sigma_x=au.FWHM_to_sigma(tmaj/dpix), sigma_y=au.FWHM_to_sigma(tmin/dpix), PA=tpa)
        tapered_beam = convolve_fft(psf_windowed, kernel=kernel)
        tapered_beam /= np.max(tapered_beam)

        metric = np.sum((target_beam[tapered_beam > psf_cutoff] - tapered_beam[tapered_beam > psf_cutoff])**2)

        return metric

    print("Calculating uvtaper parameter")
    x0 = (target_bmaj, target_bmin, target_bpa)
    res = optimize.minimize(beam_chi2, x0=x0, method="Nelder-Mead")
    uvtaper = [str(res.x[0]) + "arcsec", str(res.x[1]) + "arcsec", str(res.x[2]) + "deg"]
    print(f"Done. Best-fit uvtaper parameter: " + str(uvtaper))

    # check JvM epsilon
    print("Calculating the resulting beam shape after uvtaper")
    for ext in [".psf", ".pb", ".sumwt"]:
        shutil.rmtree(imagename + ext)
    casatasks.tclean(
        vis=vis,
        imagename=imagename,
        specmode=specmode,
        weighting=weighting,
        uvtaper=uvtaper,
        cell=cellsize,
        imsize=imsize,
        calcres=False,
        calcpsf=True,
        **tclean_kwargs
    )
    print("Done.")

    ia.open(imagename + ".psf")
    psf_tapered = ia.getregion().squeeze()
    ia.close()

     # fetch beam info
    bmaj, bmin, bpa = get_beam_info(imagename + ".psf")
    print(
        "Restoring beam shape after uvtaper: {:.4f} arcsec x {:.4f} arcsec (P.A. = {:.4f} deg)".format(
            bmaj, bmin, bpa
        )
    )

    xx, yy = get_xy_2D_coord(psf_tapered, imsize)
    target_beam = Gaussian2D(xx, yy, sigma_x=au.FWHM_to_sigma(target_bmaj/dpix), sigma_y=au.FWHM_to_sigma(target_bmin/dpix), PA=target_bpa)
    eps = calc_JvM_epsilon(xx, yy, target_beam_array=target_beam, tapered_psf_array=psf_tapered)
    if np.abs(eps - 1.0) > 0.1:
        print("Warning: dirty psf is deviated from the CLEAN beam by > 10%")

    return uvtaper


    


In [4]:
incl = disk_dict["MWC_480"]["incl"]
PA = disk_dict["MWC_480"]["PA"]
target_bmaj = 0.32

uvtaper = calc_uvtaper(
    vis=vis,
    imagename=imagename,
    field="MWC_480",
    specmode="mfs",
    weighting=weighting,
    robust=robust,
    cellsize=cellsize,
    target_beam=(target_bmaj, target_bmaj * np.cos(np.deg2rad(incl)), PA),
)
uvtaper

Starting to estimate the uvtaper which achieve the target beam of 0.3200 arcsec x 0.2556 arcsec (P.A. = 148.0000 deg)
Pixel size: 0.01arcsec
Image size: 3456
Calculating and fitting PSF with the original weighting scheme (weighting = briggs, robust = 0.5)



0%....10....20....30....40....50....60....70....80....90....100%


Done.
Restoring beam shape: 0.3146 arcsec x 0.2074 arcsec (P.A. = 8.2741 deg)
Windowing out the central region for computational efficiency
Image size of windowed psf: 1280
Calculating uvtaper parameter
Done. Best-fit uvtaper parameter: ['0.20017929935271894arcsec', '0.0009954056306741878arcsec', '301.6940457624971deg']
Calculating the resulting beam shape after uvtaper



0%....10....20....30....40....50....60....70....80....90....100%


Done.
Restoring beam shape after uvtaper: 0.3367 arcsec x 0.2608 arcsec (P.A. = -17.1913 deg)
JvM epsilon: 0.8238


['0.20017929935271894arcsec',
 '0.0009954056306741878arcsec',
 '301.6940457624971deg']

In [ ]:
# calculate uvtaper in klambda; see Eq. 27/28 in Czekala+21
a = 1e-3 * (3600 * 180 / np.pi) * 4 * np.log(2) / (np.pi * res.x[0])
b = 1e-3 * (3600 * 180 / np.pi) * 4 * np.log(2) / (np.pi * res.x[1])
phi = res.x[2]
uvtaper = [f"{a}klambda", f"{b}klambda", f"{phi}deg"]

In [6]:
rms = utils.calc_sensitivity(vis, cellsize="0.02arcsec", imsize=imsize, weighting="briggs", robust=0.5)
rms

Theoretical estimates of image sensitivity
# ./data/measurement_set/MWC_480_continuum.ms/
# specmode = 'mfs'
# spw = ''
(True, 1.3184961301317944e-05, 1.1354272877371416)
# Briggs sensitivity with robust = 0.5: 1.3184961301317944e-05 Jy
# Relative to natural weighting: 1.1354272877371416
### Averaged sensitivity over 1 measurement sets: 1.3184961301317944e-05 Jy


1.3184961301317944e-05

In [8]:
# tclean
scales = [0, 10, 30]
deconvolver = "mtmfs"
nterms = 2

# modified from MWC480_reduction_v2.py
mask_semimaj = 2.5
disk_mask = "ellipse[[{:s}, {:s}], [{:.1f}arcsec, {:.1f}arcsec], {:.1f}deg]".format(
	"04h58m46.27936s", "+29d50m36.398586s", mask_semimaj, mask_semimaj * np.cos(np.deg2rad(incl)), PA
)

rms = utils.calc_sensitivity(vis, cellsize=cellsize, imsize=imsize, weighting=weighting, uvtaper=uvtaper)

for ext in [".alpha*", ".image*", ".mask", ".model*", ".pb*", ".psf*", ".residual*", ".sumwt*"]:
    os.system("rm -r " + imagename + ext)
    
casatasks.tclean(
    vis=vis,
    imagename=imagename,
    imsize=imsize,
    cell=cellsize,
    specmode="mfs",
    deconvolver=deconvolver,
    scales=scales,
    weighting=weighting,
    nterms=nterms,
    niter=100000,
    threshold=f"{rms*4}Jy",
    usemask="user",
    mask=disk_mask,
    uvtaper=uvtaper,
    # nsigma=3.0
)

Theoretical estimates of image sensitivity
# ./data/measurement_set/MWC_480_continuum.ms/
# specmode = 'mfs'
# spw = ''
# uvtaper = [0.28209324790806867arcsec, 0.2250547197437648arcsec, 138.5533403707148deg]
(True, 1.7363699031228853e-05, 1.4952806645053645)
# Briggs sensitivity with robust = 0.5: 1.7363699031228853e-05 Jy
# Relative to natural weighting: 1.4952806645053645
### Averaged sensitivity over 1 measurement sets: 1.7363699031228853e-05 Jy



0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%

0%....10....20....30....40....50....60....70....80....90....100%


{'cleanstate': 'running',
 'cyclefactor': 1.0,
 'cycleiterdone': 0,
 'cycleniter': 100000,
 'cyclethreshold': 3.470542424111045e-06,
 'interactiveiterdone': 5317,
 'interactivemode': False,
 'interactiveniter': 0,
 'interactivethreshold': 0.0,
 'iterdone': 5317,
 'loopgain': 0.10000000149011612,
 'maxpsffraction': 0.800000011920929,
 'maxpsfsidelobe': 0.03709821030497551,
 'minpsffraction': 0.05000000074505806,
 'niter': 100000,
 'nmajordone': 5,
 'nsigma': 0.0,
 'stopcode': 2,
 'summarymajor': array([   0,  108, 1167, 5167, 5317]),
 'summaryminor': {0: {0: {0: {'iterDone': [108.0, 1059.0, 4000.0, 150.0],
     'peakRes': [0.00711461529135704,
      0.0003556274459697306,
      0.0004322105669416487,
      6.941087485756725e-05],
     'modelFlux': [0.4143482446670532,
      0.5303595662117004,
      0.5383681058883667,
      0.5384252071380615],
     'cycleThresh': [0.0071335299871861935,
      0.0003557256131898612,
      6.945479253772646e-05,
      6.945479253772646e-05]}}}},
 'thres

In [21]:
rms

1.7363699031228853e-05

/bin/bash: carta: コマンドが見つかりません
